In [9]:
# Clear any existing environment variables for a clean test
import os

# Clear MLflow-related environment variables
env_vars_to_clear = ['MODEL_ALIAS', 'MODEL_NAME', 'REGISTERED_MODEL_NAME', 'MLFLOW_TRACKING_URI']
for var in env_vars_to_clear:
    if var in os.environ:
        del os.environ[var]
        print(f"Cleared {var}")

print("Environment cleared. Ready for fresh .env loading.")

Cleared MODEL_ALIAS
Cleared REGISTERED_MODEL_NAME
Cleared MLFLOW_TRACKING_URI
Environment cleared. Ready for fresh .env loading.


In [10]:
import os
import mlflow

from utils import load_environment

# Fix: .env file is in parent directory, not in notebooks folder
env_file = os.getenv("ENV_FILE", "../.env")  # Changed from ".env" to "../.env"
load_environment(env_file)

MODEL_NAME = os.getenv("REGISTERED_MODEL_NAME")
MODEL_ALIAS = os.getenv("MODEL_ALIAS")
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print(MODEL_ALIAS)

print(f"Attempting to load model {MODEL_NAME} with alias {MODEL_ALIAS}")
# Try to load model using alias first
model_uri = f"models:/{MODEL_NAME}@{MODEL_ALIAS}"
model = mlflow.sklearn.load_model(model_uri)
print("✅ Successfully loaded MLflow model")

[ENV] Loaded environment from ../.env
candidate
Attempting to load model fraud-detection-model with alias candidate


✅ Successfully loaded MLflow model


In [11]:
# Debug: Check what's happening with environment loading
import os
print(f"Current working directory: {os.getcwd()}")
print(f"ENV_FILE environment variable: {os.getenv('ENV_FILE', 'Not set')}")

# Check if .env file exists and what it contains
env_file = os.getenv("ENV_FILE", "../.env")  # Changed to point to parent directory
print(f"Looking for env file: {env_file}")

if os.path.exists(env_file):
    print(f"✅ {env_file} exists")
    with open(env_file, 'r') as f:
        for line_num, line in enumerate(f, 1):
            if 'MODEL_ALIAS' in line:
                print(f"Line {line_num}: {line.strip()}")
else:
    print(f"❌ {env_file} does not exist")

# Check environment variable before and after loading
print(f"MODEL_ALIAS before loading: {os.getenv('MODEL_ALIAS', 'Not set')}")

from utils import load_environment
load_environment(env_file)

print(f"MODEL_ALIAS after loading: {os.getenv('MODEL_ALIAS', 'Not set')}")

Current working directory: /Users/carlychinsekyi/Downloads/GitHub/mlops-fyp/notebooks
ENV_FILE environment variable: Not set
Looking for env file: ../.env
✅ ../.env exists
Line 41: MODEL_ALIAS=candidate
MODEL_ALIAS before loading: candidate
[ENV] Loaded environment from ../.env
MODEL_ALIAS after loading: candidate
